In [1]:
import os
os.environ["XDG_CACHE_HOME"] = "/home/olab/tomerronen1/xdg_cache/"
from bert_score import BERTScorer

force_recalculation = False
lang = "et"
rescale_with_baseline = (lang != "tr")
if lang == "en":
    scorer = BERTScorer(model_type="microsoft/deberta-xlarge-mnli", lang="en", rescale_with_baseline=True)
else:
    scorer = BERTScorer(lang=lang, rescale_with_baseline=rescale_with_baseline)

scorer.score(['a',], ['a'])

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(tensor([1.0000]), tensor([1.0000]), tensor([1.0000]))

In [2]:
from pathlib import Path
import jsonlines
dump_dir = Path("/home/olab/tomerronen1/git_repos/last_projects_playground/confidence_estimation/openai_dump")
for subdir in dump_dir.iterdir():
    results_dirs = [subdir]
    bt_dump_dir = subdir / "backtranslation" / "dump"
    if bt_dump_dir.exists():
        results_dirs.append(bt_dump_dir)
    
    for results_dir in results_dirs:
        if (f"to_{lang}" in str(results_dir)) or ((f"{lang}_to" in str(results_dir)) and ("backtranslation" in str(results_dir))):
            with jsonlines.open(results_dir / "metrics.jsonl", 'r') as reader:
                metrics_list = list(reader.iter())
            with jsonlines.open(results_dir / "sentences.jsonl", 'r') as reader:
                sentences_list = list(reader.iter())

            if force_recalculation or ("bertscore_f1" not in metrics_list[0]):
                print(f"Calculating BertScore for {results_dir}\n")

                candidates = [sentences["pred_text"] for sentences in sentences_list]
                references = [sentences["target_text"] for sentences in sentences_list]
                precisions, recalls, f1s = scorer.score(candidates, references)
                precisions, recalls, f1s = precisions.tolist(), recalls.tolist(), f1s.tolist()
                metrics_list = [
                    {"bertscore_precision": precision, "bertscore_recall": recall, "bertscore_f1": f1, **metrics}
                    for precision, recall, f1, metrics
                    in zip(precisions, recalls, f1s, metrics_list)
                ]
                
                with jsonlines.open(results_dir / "metrics.jsonl", 'w') as writer:
                    writer.write_all(metrics_list)


Calculating BertScore for /home/olab/tomerronen1/git_repos/last_projects_playground/confidence_estimation/openai_dump/wmt19_en_to_et__text-davinci-002__200_examples

